# Chapter 8. 차원 축소

많은 경우 머신러닝 문제는 훈련 샘플 각각이 수천 심지어 수백만 개의 특성을 가지고 있습니다. 이는 훈련을 느리게 할 뿐만 아니라 앞으로 보게되겠지만 좋은 솔루션을 찾기 어렵게 만듭니다. 이런 문제를 종종 **차원의 저주**(curse of dimensionality)라고 합니다.

다행히도 실전 문제에서는 특성 수를 크게 줄여서 불가능한 문제를 가능한 범위로 변경할 수 있는 경우가 많습니다. 분명 차원을 축소시키면 일부 정보가 유실됩니다. (JPEG로 이미지를 압축하면 품질이 감소되는 것처럼) 그래서 훈련 속도가 빨라질 수는 있지만 시스템의 성능이 조금 나빠질 수 있습니다. 또한 직업 파이프라인이 조금 더 복잡하게 되고 유지 관리가 어려워집니다. 그러므로 차원 축소를 고려하기 전에 훈련이 너무 느린지 먼저 원본 데이터로 시스템을 훈련시켜봐야 합니다. 그러나 어떤 경우에는 훈련 데이터의 차원을 축소시키면 잡음이나 불필요한 세부 사항을 걸러내므로 성능을 높일 수 있습니다.

훈련 속도를 높이는 것 외에 차원 축소는 데이터 시각화에도 아주 유용합니다. 차원 수를 둘로 줄이면 고차원 훈련 세트를 하나의 그래프로 그릴 수 있고 군집 같은 시각적인 패턴을 감지해 중요한 통찰을 얻는 경우가 많습니다. 

이 장에서는 차원의 저주에 대해 논의하고 고차원의 공간에서 어떤 일이 일어나고 있는지 알아보도록 하겠습니다. 그런 다음 차원 축소에 사용되는 두 가지 주요 접근 방법을 소개하겠습니다. (투영(projection)과 매니폴드 학습(manifold learning)) 그리고 가장 인기있는 차원 축소 기법인 PCA, 커널 PCA, LLE를 다루도록 하겠습니다.

## 1. 차원의 저주

우리는 3차원의 세계에서 살고있어서 고차원의 공간을 직관적으로 상상하기 어렵습니다. 고차원에서는 많은 것이 상당히 다르게 작동합니다. 예를 들어 단위 면적 안에 있는 점을 무작위로 선택한다면 경계선에서 0.001 이내에 위치할 가능성은 0.4%입니다. 하지만 10,000차원의 단위 면적을 가진 초입방체에서는 이 가능성이 99.99999%보다 커집니다. 고차원 초입방체에 있는 대다수의 점은 경계와 매우 가까이 있습니다.

고차원의 데이터는 예측을 위해 많은 외삽(extrapolation)을 해야하기 때문에 저차원일때보다 예측이 더 불안정합니다. 간단히 말해 *훈련 세트의 차원이 클수록 과대 적합의 위험이 커집니다.*

이론적으로 차원의 저주를 해결하는 해결책 하나는 훈련 샘플의 밀도가 충분히 높아질 때 까지 훈련 세트의 크기를 키우는 것입니다. 불행하게도 실제로는 일정 밀도에 도달하기 위해 필요한 훈련 샘플 수는 차원 수가 커짐에 따라 기하급수적으로 늘어납니다. 특성 수가 단 100개라고 생각해도 모든 차원에 걸쳐 균일하게 퍼져있다고 가정하고 훈련 샘플을 서로 평균 0.1 이내에 위치시키려면 관측 가능한 우주에 있는 원자 수를 모두 합친 것보다 더 많은 훈련 샘플을 모아야 합니다. (오우..)



## 2. 차원 축소를 위한 접근 방법

구체적인 차원 축소 알고리즘을 알아보기 전에 차원을 감소시키는 두 가지 주요한 접근법인 투영(projection)과 매니폴드 학습(manifold learning)을 알아봅시다.

### 2.1 투영

대부분의 실전 문제는 훈련 샘플이 모든 차원에 걸쳐 균일하게 퍼져 있지 않습니다. 많은 특성은 거의 변화가 없는 반면, 다른 특성들은 서로 강하게 연관되어 있습니다. 결과적으로 모든 훈련 샘플이 사실 고차원 공간 안의 저차원 **부분 공간**(subspace)에 놓여있습니다. 

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig1.png)

모든 훈련 샘플이 거의 평면 형태로 놓여있습니다. 이것이 고차원(3D) 공간에 있는 저차원(2D) 부분 공간입니다. 여기서 모든 훈련 샘플을 이 부분 공간에 수직으로(즉, 샘플과 평면 사이의 가장 짧은 직선을 따라) 투영하면 아래 그림과 같은 2D 데이터셋을 얻습니다. 데이터 셋의 차원을 3D에서 2D로 줄인 것입니다. 각 축은 새로운 특성 $z_1$과 $z_2$에 대응됩니다.

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig2.png)

그러나 차원 축소에 있어 투영이 언제나 최선의 방법은 아닙니다. 많은 경우 아래 그림처럼 **스위스 롤**(swiss roll) 데이터셋처럼 부분 공간이 뒤틀리거나 휘어 있기도 합니다. 

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig3.png)

그냥 평면에 투영시키면 아래 그림의 왼쪽처럼 스위스 롤의 층이 서로 뭉개집니다. 하지만 우리가 원하는 것은 스위스 롤을 펼쳐서 오른쪽 그림처럼 2D 데이터 셋을 얻는 것입니다. 

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig4.png)

### 2.2 매니폴드 학습

스위스 롤은 2D의 **매니폴드**의 한 예입니다. 간단히 말해 2D 매리폴드는 고차원 공간에서 휘어지거나 뒤틀린 2D 모댱입니다. 더 일반적으로 $d$차원 매니폴드는 국부적으로 $d$차원 초평면으로 보일 수 있는 $n$차원 공간의 일부입니다.($d < n$). 스위스 롤의 경우에는 $d=2$이고, $n=3$입니다. 국부적으로는 2D 평면처럼 보이지만 3차원으로 말려 있습니다. 

많은 차원 축소 알고리즘이 훈련 샘플이 놓여있는 **매니 폴드**를 모델링 하는 식으로 작동합니다. 이를 **매니폴드 학습**(manifold learning)이라고 합니다. 이는 대부분 실제 고차원 데이터셋이 더 낮은 저차원 매니폴드에 가깝게 놓여 있다는 **매니폴드 가정**(manifold assumption) 또는 **매니폴드 가설**(manifold hypothesis)에 근거합니다. 경헙적으로도 이런 가정은 매우 자주 발견됩니다.

여기에서도 MNIST 데이터셋으로 생각해보겠습니다. 전체 손글씨 숫자 이미지는 어느 정도 비슷한 면이 있습니다. 선으로 연결되어 있고 경계는 흰색이고 어느정도 중앙에 있습니다. 무작위로 생성된 이미지라면 그 중 아주 적은 일부만 손글씨 숫자처럼 보일 것입니다. 다시 말해 숫자 이미지를 만들 때 가능한 자유도는 아무 이미지나 생성할 때의 자유도보다 훨씬 낮습니다. 이런 제약은 데이터 셋을 저차원의 매니폴드로 압축할 수 있도록 도와줍니다.

매니폴드 가정은 종종 암묵적으로 다른 가정과 병행되곤 합니다. 바로 처리해야할 작업이(예를 들어 분류나 회귀)이 저아춴의 매니폴드 공간에 표현되면 더 간단해질 것이란 가정입니다. 아래 그림의 두번째 행의 경우에는 결정 경계가 $x_1=5$에 놓여있습니다. 이 결정 경계는 3D 공간에서는 매우 단순합니다. 하지만 펼쳐진 매니폴드에서는 결정 경계가 더 복잡해졌습니다. 

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig5.png)

그러나 이런 가정이 항상 유효하지는 않습니다. 

요약하자면 모델을 훈련시키기 전에 훈련 세트의 차원을 감소시키면 훈련 속도는 빨자지지만 항상 더 낫거나 간단한 솔루션이 되는 것은 아닙니다. 이는 전적으로 데이터 셋에 달렸습니다.


## 3. PCA

**주성분 분석**(Principal Component Analysis)(PCA)는 가장 인기있는 차원 축소 알고리즘입니다. 먼저 데이터에 가장 가까운 초평면(hypterplane)을 정의한 다음, 데이터를 이 평면에 투영시킵니다.

### 3.1 분산 보존

저차원의 초평면에 훈련 세트를 투영하기 전에 먼저 올바른 초평면을 선택해야 합니다. 예를 들어 아래 왼쪽 그래프는 간단한 2D 데이터셋이 세 개의 축과 함께 표현되어 있습니다. 오른쪽 그래프는 데이터셋이 각 축에 투영된 결과입니다. 여기서 볼 수 있듯이 실선에 투영된 것은 분산을 최대로 보존하는 반면, 점선에 투영된 것은 분산을 매우 적게 유지하고 있습니다. 가운데 파선에 투영된 것은 분산을 중간 정도로 유지하고 있습니다. 

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig6.png)

다른 방향으로 투영하는 것보다 분산이 최대로 보존되는 축을 선택하는 것이 정보가 가장 적게 손실되므로 합리적으로 보입니다. 이 선택을 다른 방식으로 설명하면 원본 데이터셋과 투영된 것 사이에 평균 제곱 거리를 최소화하는 축입니다. 이 방식이 PCA를 더 간단하게 설명할 수 있습니다.


### 3.2 주성분

PCA는 훈련 세트에 *분산이 최대인 축*을 찾습니다. 위 그림에서는 실선이 됩니다. 또한 첫 번째 축에 직교하고 남은 분산을 최대한 보존하는 두번째 축을 찾습니다. 이 2D 예제에서는 선택의 여지가 없습니다. 즉, 점선이 됩니다. 고차원 데이터셋이라면 PCA는 이전의 두 축에 직교한느 세번째 축을 찾으며 데이터셋에 있는 차원의 수만큼 네 번째, 다섯번째, ... 축을 찾습니다.

$i$번째 축을 정의하는 단위 벡터를 $i$번째 **주성분**(Principal Component)(PC)라고 부릅니다. 위 그림에서 첫번째 PC는 $c_1$이고, 두번째 PC는 $c_2$입니다. 

#### 훈련 세트의 주성분을 찾는 방법

그럼 훈련 세트의 주성분을 어떻게 찾을까요? 다행히 **특잇값 분해**(Singular Value Decomposition)(SVD)라는 표준 행렬 분해 기술이 잇어서 훈련 세트 행렬 X를 세 개의 행렬의 점 곱인 $U \cdot \sum \cdot V^T$로 분해할 수 있습니다. 여기서 찾고자 하는 모든 주성분이 $V$에 아래와 같이 담겨 있습니다. 

![img](https://t1.daumcdn.net/cfile/tistory/99ECE34A5B646BF22C)

아래 코드는 넘파이의 `svd()` 함수를 사용해 훈련 세트의 모든 주성분을 구한 후 처음 두개의 PC를 추출합니다.

```python
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]
```

PCA는 데이터셋의 평균이 0이라고 가정합니다. 앞으로 볼 사이킷런의 PCA 파이썬 클래스는 이 작업을 대신 처리해줍니다. 그러나 앞의 코드처럼 PCA를 직접 구현하거나 다른 라이브러리를 사용한다면 먼저 데이터를 원점에 맞추는 것을 잊어서는 안됩니다. 